In [1]:
import numpy as np
import csv 
import time


In [2]:
np.random.seed(1234)


In [3]:
def randomize() -> None:
    np.random.seed(time.time())

In [7]:
RND_MEAN = 0
RND_STD = 0

LEARNING_RATE = 0.001

In [5]:
def abalone_exec(epoch_count:int=10, mb_size:int=10, report:int=1) -> None:
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)
    


In [6]:
def load_abalone_dataset() -> None:
    with open("~/workspace/dataset/abalone/abalone.data.csv") as f:
        reader = csv.reader(f)
        rows = []
        for idx, row in enumerate(reader):
            if idx == 0:
                print(row)
                continue

            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for idx, row in enumerate(rows):
        if row[0] == 'I': data[idx, 0] = 1
        if row[0] == 'M': data[idx, 1] = 1
        if row[0] == 'F': data[idx, 2] = 1
        data[idx, 3:] = row[1:]
    


In [ ]:
def init_model() -> None:
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [ ]:
def train_and_test(epoch_count:int, mb_size:int, report:int) -> None:
    setp_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
        
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print(f"Epoch {epoch+1} loss={np.mean(losses):5.3f} accuracy={np.mean(accs):5.3f}/{acc:5.3f}.")
    
    final_acc = run_test(test_x, test_y)
    print(f"\nFinal Test: final accuracy = {final_acc:5.3f}")

In [ ]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, :-output_cnt]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]